In [1]:
import tensorflow as tf
import numpy as np
from pympler.asizeof import asizeof
import os

In [2]:
checkdir = "./save"
def one_hot(s):
    arr = np.zeros(7,np.uint8)
    arr[int(s)]=1
    return arr

In [3]:
all_label = np.loadtxt('all_label.txt', dtype=np.uint8)
#one-hot encoding
print(np.shape(all_label), ", label 완료")
all_data = np.loadtxt('all_data.txt', dtype=np.uint8)

last_epoch = tf.Variable(0, tf.int32)

(35887, 7) , label 완료


In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.01)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.001, shape=shape)
    return tf.Variable(initial)

In [5]:
def conv2d(x, c_in, c_out, kernel_size = [3,3], bn = True, pad = True):
    w = weight_variable([*kernel_size, c_in, c_out])
    b = bias_variable([c_out])
    p = "SAME" if pad else "VALID"
    c = tf.nn.conv2d(x, w, [1,1,1,1], p) + b
    if bn is True:
        c = tf.contrib.layers.batch_norm(c)
    return tf.nn.relu(c)

In [6]:
def max_pool(x, kernel_size = [2,2]):
    return tf.nn.max_pool(x, [1, *kernel_size, 1], [1, *kernel_size, 1], "VALID")

In [7]:
X = tf.placeholder(tf.float32, shape = [None, 2304])
y_ = tf.placeholder(tf.float32, shape = [None, 7])

X_img = tf.reshape(X,[-1,48,48,1])

conv1 = conv2d(X_img, 1, 64)
conv2 = conv2d(conv1, 64, 64)
pool3 = max_pool(conv2)

conv4 = conv2d(pool3, 64, 128)
conv5 = conv2d(conv4, 128, 128)
conv6 = conv2d(conv5, 128, 128)
pool7 = max_pool(conv6)

conv8 = conv2d(pool7, 128, 256)
conv9 = conv2d(conv8, 256, 256)
conv10 = conv2d(conv9, 256, 256)
pool11 = max_pool(conv10)

In [8]:
flat = tf.reshape(pool11, [-1,6*6*256])

w4 = weight_variable([6*6*256, 2048])
b4 = bias_variable([2048])
layer4 = tf.nn.relu(tf.matmul(flat,w4)+b4)

w5 = weight_variable([2048, 1000])
b5 = bias_variable([1000])
layer5 = tf.nn.relu(tf.matmul(layer4,w5)+b5)

w6 = weight_variable([1000,7])
b6 = bias_variable([7])
y_conv = tf.matmul(layer5,w6)+b6

y_loss= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.GradientDescentOptimizer(1e-2).minimize(y_loss)
y_class = tf.argmax(y_conv,1)

In [9]:


num_epoch = 75
batch_size = 16

displayper = 20

m=np.shape(all_data)[0]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(100):
        for j in range(10000):
            if len(all_data) <= j*batch_size:
                break
                
            batch_data = all_data[j*batch_size:(j+1)*batch_size]
            batch_label = all_label[j*batch_size:(j+1)*batch_size]
            feed_train = {X: batch_data, y_: batch_label}
        
            sess.run(train_step, feed_dict = feed_train)
            
            if j % displayper == 0:
                print(epoch, "th epoch, ", j, "th batch",sess.run(y_loss, feed_dict=feed_train))

0 th epoch,  0 th batch 1.88531
0 th epoch,  20 th batch 1.5988
0 th epoch,  40 th batch 1.59218
0 th epoch,  60 th batch 1.6475
0 th epoch,  80 th batch 1.56554
0 th epoch,  100 th batch 1.63134
0 th epoch,  120 th batch 1.55133
0 th epoch,  140 th batch 1.64924
0 th epoch,  160 th batch 1.60944
0 th epoch,  180 th batch 1.58627
0 th epoch,  200 th batch 1.43303
0 th epoch,  220 th batch 1.42619
0 th epoch,  240 th batch 1.36356
0 th epoch,  260 th batch 1.30206
0 th epoch,  280 th batch 1.37138
0 th epoch,  300 th batch 1.51208
0 th epoch,  320 th batch 1.30178
0 th epoch,  340 th batch 1.26624
0 th epoch,  360 th batch 1.1909
0 th epoch,  380 th batch 1.32524
0 th epoch,  400 th batch 1.24332
0 th epoch,  420 th batch 1.26535
0 th epoch,  440 th batch 1.41811
0 th epoch,  460 th batch 1.31285
0 th epoch,  480 th batch 1.23146
0 th epoch,  500 th batch 1.2455
0 th epoch,  520 th batch 0.860223
0 th epoch,  540 th batch 1.06899
0 th epoch,  560 th batch 1.08536
0 th epoch,  580 th bat

KeyboardInterrupt: 